# Preenchendo Valores Nulos/NaN

### Mas por quê?

1. **Evita erros de execução**: A maioria dos algoritmos de ML não aceitam NaN como entrada e lançarão erros ou falharão.

2. **Garante estatísticas corretas**: Métricas e transformações são calculadas sobre todos os valores disponíveis. NaN contamina esses cálculos, resultando em estatísticas enviesadas.

3. **Mantém a qualidade do aprendizado**:
    
    1. Se simplesmente omitir essas linhas sem critério, pode desequilibrar a distribuição de classes ou remover casos críticos.
    
    2. Imputação (substituir NaN por valores plausíveis) ou modelagem dos padrões de ausência ajudam o modelo a aprender de forma mais robusta.

4. **Reduz viés e overfiting**: Dados faltantes não são aleatórios em muitos casos. Se ignorá-los, cria um viés: o modelo vê apenas casos "completos", que podem não representar toda a realidade.

In [1]:
import pandas as pd

car_sales_missing_data_df = pd.read_csv("./dataset/car-sales-extended-missing-data.csv")

# visualização da quantidade total de valores NaN
car_sales_missing_data_df.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

## Como Preencher Valores Nulos/NaN com Pandas?

In [2]:
# Preenche coluna "Make"
car_sales_missing_data_df.loc[:, "Make"] = car_sales_missing_data_df["Make"].fillna("missing")

# Preenche coluna "Colour"
car_sales_missing_data_df.loc[:, "Colour"] = car_sales_missing_data_df["Colour"].fillna("missing")

# Preenche coluna "Odometer (KM)"
car_sales_missing_data_df.loc[:, "Odometer (KM)"] = car_sales_missing_data_df["Odometer (KM)"].fillna(car_sales_missing_data_df["Odometer (KM)"].median())

#Preenche coluna "Doors"
car_sales_missing_data_df.loc[:, "Doors"] = car_sales_missing_data_df["Doors"].fillna(int(car_sales_missing_data_df["Doors"].mode().values[0]))

### Imputação 

A imputação de dados é o processo de substituir valores faltantes (NaN) por outros valores, de modo a manter a integridade do seu conjunto de dados antes de treinar um modelo. Há duas categorias principais de estratégias:

1. **Imputação por valor constante ("*Placeholder*")**: Você preenche todos os NaN de uma coluna com um mesmo valor genérico, por exemplo "*missing*" (para variáveis categóricas) ou *0* (para numéricas). **Quando usar?** Se a ausência em si pode ser significativa.

2. **Imputação por estatísticas descritivas**:

    1. por **Média**: Substitui NaN pelo valor médio da coluna. **Quando usar?** Dados numéricos aproximadamente simétricos, sem outliers fortes.


    3. por **Mediana**: Usa o valor “do meio” ao ordenar os dados. **Quando usar?** Distribuições enviesadas ou com outliers.
      
    4. por **Moda**: Usa o valor que mais aparece na coluna. **Quando usar?** a) Variáveis categóricas ou discretas (ex.: número de portas). b) Quando faz sentido assumir que os casos faltantes provavelmente seguem o padrão mais comum.       

In [3]:
# Vereficar novamente quantidade de valores Nan em df
car_sales_missing_data_df.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

In [4]:
car_sales_missing_data_df.dropna(inplace = True)

### Listwise Deletion

Em vez de preencher os NaN, é descartado toda a linha/registro que os contém.

#### Quando usar?

1. **Proporção pequena de faltantes**

Se apenas uma fração desprezível (< 5 %) dos seus casos tem NaN, remover essas linhas não vai afetar muito o tamanho nem a representatividade do seu dataset.

2. **Missing Completely At Random (MCAR)**

Só é seguro descartar dados quando a ausência é completamente aleatória — isto é, não está correlacionada com outra variável ou com o próprio valor faltante. Caso contrário, você introduz viés.

3. **Simplicidade**

Se não é utilizado em cálculos, pipelines de imputação nem suposições sobre distribuição. Ótimo para protótipos ou análises exploratórias.

---


In [5]:
# Features
x = car_sales_missing_data_df.drop("Price", axis = 1)

# Target
y = car_sales_missing_data_df["Price"]

In [6]:
# Converte valores categóricos em binários

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder = "passthrough")

transformed_x = transformer.fit_transform(car_sales_missing_data_df)

transformed_x

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        3.54310e+04, 1.53230e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        1.92714e+05, 1.99430e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        8.47140e+04, 2.83430e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        6.66040e+04, 3.15700e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.15883e+05, 4.00100e+03],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.48360e+05, 1.27320e+04]], shape=(950, 16))

In [7]:
pd.DataFrame(transformed_x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0,15323.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0,19943.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0,28343.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0,13434.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0,14043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0,32042.0
946,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,155144.0,5716.0
947,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0,31570.0
948,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,215883.0,4001.0
